Ось SQL-скрипт для створення та заповнення бази даних **"Навчання персоналу"**, що містить дві таблиці:  

- **Personnel** (особовий склад)  
- **Training** (навчальні курси)  

### **1. Створення бази даних та таблиць**
```sql
-- Створення бази даних
CREATE DATABASE TrainingDB;
\c TrainingDB;

-- Таблиця особового складу (Personnel)
CREATE TABLE Personnel (
    personnel_id SERIAL PRIMARY KEY,
    full_name VARCHAR(100) NOT NULL,
    rank VARCHAR(50) NOT NULL,
    position VARCHAR(100) NOT NULL,
    unit VARCHAR(100) NOT NULL
);

-- Таблиця навчальних курсів (Training)
CREATE TABLE Training (
    training_id SERIAL PRIMARY KEY,
    course_name VARCHAR(100) NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    instructor VARCHAR(100) NOT NULL,
    personnel_id INT REFERENCES Personnel(personnel_id) ON DELETE CASCADE
);
```

---

### **2. Заповнення таблиць тестовими даними**
```sql
-- Додавання персоналу
INSERT INTO Personnel (full_name, rank, position, unit) VALUES
('Іван Петров', 'Капітан', 'Командир роти', 'Батальйон A'),
('Андрій Сидоренко', 'Лейтенант', 'Офіцер зв’язку', 'Батальйон B'),
('Олександр Шевченко', 'Майор', 'Заступник командира', 'Батальйон A'),
('Дмитро Коваленко', 'Сержант', 'Механік', 'Батальйон C'),
('Сергій Гончар', 'Підполковник', 'Начальник штабу', 'Батальйон D');

-- Додавання навчальних курсів
INSERT INTO Training (course_name, start_date, end_date, instructor, personnel_id) VALUES
('Стратегічне планування', '2024-01-10', '2024-01-20', 'Полковник Іванов', 1),
('Тактична медицина', '2024-02-05', '2024-02-15', 'Майор Козлов', 2),
('Кібербезпека', '2024-03-01', '2024-03-10', 'Капітан Бондар', 3),
('Навігація в бойових умовах', '2024-04-15', '2024-04-25', 'Сержант Павленко', 4),
('Аналітичні системи', '2024-05-20', '2024-05-30', 'Лейтенант Савченко', 5);
```

---

### **3. 20 SQL-запитів до бази даних**
#### **3.1. Простий вибір даних**
1. **Отримати список всіх військовослужбовців**
```sql
SELECT * FROM Personnel;
```
2. **Вивести імена всіх військових та їхні звання**
```sql
SELECT full_name, rank FROM Personnel;
```
3. **Вибрати всіх військовослужбовців з підрозділу "Батальйон A"**
```sql
SELECT * FROM Personnel WHERE unit = 'Батальйон A';
```
4. **Знайти всі навчальні курси, що тривають більше 10 днів**
```sql
SELECT * FROM Training WHERE end_date - start_date > 10;
```
5. **Сортувати список персоналу за званням у порядку спадання**
```sql
SELECT * FROM Personnel ORDER BY rank DESC;
```

#### **3.2. Використання JOIN**
6. **Отримати список курсів разом з інформацією про військовослужбовців**
```sql
SELECT t.course_name, t.start_date, t.end_date, p.full_name, p.rank 
FROM Training t
JOIN Personnel p ON t.personnel_id = p.personnel_id;
```
7. **Знайти, які курси проходив військовий "Іван Петров"**
```sql
SELECT t.course_name FROM Training t
JOIN Personnel p ON t.personnel_id = p.personnel_id
WHERE p.full_name = 'Іван Петров';
```
8. **Отримати список курсів, що проводив "Полковник Іванов"**
```sql
SELECT course_name FROM Training WHERE instructor = 'Полковник Іванов';
```
9. **Отримати кількість курсів, які пройшов кожен військовий**
```sql
SELECT p.full_name, COUNT(t.training_id) AS num_courses
FROM Personnel p
LEFT JOIN Training t ON p.personnel_id = t.personnel_id
GROUP BY p.full_name;
```

#### **3.3. Використання агрегатних функцій**
10. **Підрахувати кількість військових у кожному підрозділі**
```sql
SELECT unit, COUNT(*) AS num_personnel FROM Personnel GROUP BY unit;
```
11. **Визначити середню тривалість курсів**
```sql
SELECT AVG(end_date - start_date) AS avg_duration FROM Training;
```
12. **Знайти курс з найменшою тривалістю**
```sql
SELECT course_name FROM Training ORDER BY (end_date - start_date) ASC LIMIT 1;
```
13. **Знайти максимальну кількість курсів, що пройшла одна особа**
```sql
SELECT MAX(num_courses) FROM (
    SELECT COUNT(training_id) AS num_courses FROM Training GROUP BY personnel_id
) AS temp;
```

#### **3.4. Використання підзапитів**
14. **Отримати військових, які ще не проходили жодного курсу**
```sql
SELECT * FROM Personnel WHERE personnel_id NOT IN (SELECT personnel_id FROM Training);
```
15. **Отримати назву курсу, що проходив військовий із найвищим званням**
```sql
SELECT t.course_name FROM Training t
JOIN Personnel p ON t.personnel_id = p.personnel_id
WHERE p.rank = (SELECT MAX(rank) FROM Personnel);
```

#### **3.5. Використання обмежень**
16. **Отримати 3 найновіші курси**
```sql
SELECT * FROM Training ORDER BY start_date DESC LIMIT 3;
```
17. **Перевірити, чи існують курси, що починаються після 2024-06-01**
```sql
SELECT EXISTS (SELECT 1 FROM Training WHERE start_date > '2024-06-01');
```

#### **3.6. Оновлення та видалення**
18. **Оновити звання військовослужбовця "Іван Петров"**
```sql
UPDATE Personnel SET rank = 'Майор' WHERE full_name = 'Іван Петров';
```
19. **Видалити всі курси, що завершились до 2024-03-01**
```sql
DELETE FROM Training WHERE end_date < '2024-03-01';
```
20. **Видалити військовослужбовця разом із його навчальними записами**
```sql
DELETE FROM Personnel WHERE personnel_id = 1;
```

---

## **4. Висновки**
- База даних **"Навчання персоналу"** містить інформацію про особовий склад та їхні навчальні курси.
- Запити дозволяють вибирати, аналізувати, оновлювати та видаляти дані.
- Використання `JOIN`, `GROUP BY`, `ORDER BY` та підзапитів дозволяє отримати детальну інформацію про навчальний процес.
- **Автоматизація управління персоналом та навчанням значно підвищує ефективність військової аналітики.**